# About this notebook



---



---



In [41]:
# Mount YOUR google drive. You'll need to "Add shortcut to Drive" for our shared folder for it to show up here.
# Use the URL shown below in the output to authorize this Colab session to access you GDrive
from google.colab import drive
drive.mount('/content/drive/',force_remount=True)

# Modify this according to the path in your computer
data_dir = '/content/drive/MyDrive/SURI/Studies/2022_Wildfires/' # <-- change this to reflect the pathing in your machine

Mounted at /content/drive/


In [42]:
# Import needed packages
! pip install geopandas
! pip install geopy
! pip install -U plotly
! pip install cmcrameri
! pip install cpi
! pip install requests
!pip install adjustText

import pandas as pd
import geopandas as gpd
import geopy
from geopy.geocoders import Nominatim
from geopy.extra.rate_limiter import RateLimiter
import geopy.distance
from shapely.geometry import Point, Polygon
import csv
import math
import numpy as np
import matplotlib.pyplot as plt
import scipy 
from scipy import stats as sts
import plotly.express as px
from numpy.random import default_rng
from plotnine import *
import time
from cmcrameri import cm
import cpi
import requests 

rng = default_rng(13)

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


# Approval rates

# Get hold of the data and clean it

### Data from Goldsten (2019) obtained through the Freedom for Information Act

In [43]:
SBA_dir = data_dir + 'SBA/'

# Approved loans
df_approvals = pd.read_csv(SBA_dir + "data/sba_disaster_loan_approvals.csv.gz")

# Withdrawn loans
df_withdrawals = pd.read_csv(SBA_dir + "data/sba_disaster_loan_withdrawals.csv.gz")

# Declined loans
df_declines = pd.read_csv(SBA_dir + "data/sba_disaster_loan_declines.csv.gz")

# Remove whitespace from decline code fields
df_declines[['DECL_CODES','DECL_CODES2','DECL_CODES3','DECL_CODES4','FEMA_DECL']] = df_declines[['DECL_CODES','DECL_CODES2','DECL_CODES3','DECL_CODES4','FEMA_DECL']].apply(lambda s: s.str.strip())

# Remove whitespace from ends of state codes
df_approvals[['STATE']] = df_approvals[['STATE']].apply(lambda s: s.str.strip())
df_withdrawals[['STATE']] = df_withdrawals[['STATE']].apply(lambda s: s.str.strip())
df_declines[['STATE']] = df_declines[['STATE']].apply(lambda s: s.str.strip())

# Looking only at HOME LOANS & LOANS BELOW 200K with FEMA verified losses
cols = ['ORIGINAL_APPROVAL_AMOUNT']
mask = df_approvals[cols].applymap(lambda x: isinstance(x, (int, float)))
df_approvals[cols] = df_approvals[cols].where(mask)
df_approvals = df_approvals[df_approvals['ORIGINAL_APPROVAL_AMOUNT'].astype(float)<210000].reset_index(drop=True)
df_approvals['FEMA_DECL'] = pd.to_numeric(df_approvals['FEMA_DECL'].str.replace('DR',''),errors='coerce')

cols = ['TOT_ORIG_VER_LOSS']
mask = df_declines[cols].applymap(lambda x: isinstance(x, (int, float)))
df_declines[cols] = df_declines[cols].where(mask)
df_declines = df_declines[(df_declines['TOT_ORIG_VER_LOSS_RE'] > 100) & (df_declines['TOT_ORIG_VER_LOSS_RE'].astype(float)<210000)]

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:3326: DtypeWarning: Columns (1,2,3,7,10) have mixed types.Specify dtype option on import or set low_memory=False.
/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:3326: DtypeWarning: Columns (1,2,3,7,10,13,14,15) have mixed types.Specify dtype option on import or set low_memory=False.
/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:3326: DtypeWarning: Columns (1,2,3,7,10,13,14,15,16) have mixed types.Specify dtype option on import or set low_memory=False.


### Data from the OpenSBA portal

In [44]:
# Read
SBA_dir = data_dir + 'SBA/'
df_SBA_2008 = pd.read_csv(SBA_dir + "SBA_FY08.txt")
df_SBA_2009 = pd.read_csv(SBA_dir + "SBA_FY09.txt")
df_SBA_2010 = pd.read_csv(SBA_dir + "SBA_FY10.txt")
df_SBA_2011 = pd.read_csv(SBA_dir + "SBA_FY11.txt")
df_SBA_2012 = pd.read_csv(SBA_dir + "SBA_FY12.txt")
df_SBA_2013 = pd.read_csv(SBA_dir + "SBA_FY13.txt")
df_SBA_2014 = pd.read_csv(SBA_dir + "SBA_FY14.txt")
df_SBA_2015 = pd.read_csv(SBA_dir + "SBA_FY15.txt")
df_SBA_2016 = pd.read_csv(SBA_dir + "SBA_FY16.txt")
df_SBA_2017 = pd.read_csv(SBA_dir + "SBA_FY17.txt")
df_SBA_2018 = pd.read_csv(SBA_dir + "SBA_FY18.txt") # Not available in 2017
df_SBA_2019 = pd.read_csv(SBA_dir + "SBA_FY19.txt") # Not available in 2017

df_SBA_All = pd.DataFrame()
df_SBA_All = df_SBA_All.append(df_SBA_2008[['FEMA Disaster Number','Damaged Property State Code','Verified Loss Real Estate','Approved Amount Real Estate']])
df_SBA_All = df_SBA_All.append(df_SBA_2009[['FEMA Disaster Number','Damaged Property State Code','Verified Loss Real Estate','Approved Amount Real Estate']])
df_SBA_All = df_SBA_All.append(df_SBA_2010[['FEMA Disaster Number','Damaged Property State Code','Verified Loss Real Estate','Approved Amount Real Estate']])
df_SBA_All = df_SBA_All.append(df_SBA_2011[['FEMA Disaster Number','Damaged Property State Code','Verified Loss Real Estate','Approved Amount Real Estate']])
df_SBA_All = df_SBA_All.append(df_SBA_2012[['FEMA Disaster Number','Damaged Property State Code','Verified Loss Real Estate','Approved Amount Real Estate']])
df_SBA_All = df_SBA_All.append(df_SBA_2013[['FEMA Disaster Number','Damaged Property State Code','Verified Loss Real Estate','Approved Amount Real Estate']])
df_SBA_All = df_SBA_All.append(df_SBA_2014[['FEMA Disaster Number','Damaged Property State Code','Verified Loss Real Estate','Approved Amount Real Estate']])
df_SBA_All = df_SBA_All.append(df_SBA_2015[['FEMA Disaster Number','Damaged Property State Code','Verified Loss Real Estate','Approved Amount Real Estate']])
df_SBA_All = df_SBA_All.append(df_SBA_2016[['FEMA Disaster Number','Damaged Property State Code','Verified Loss Real Estate','Approved Amount Real Estate']])
df_SBA_All = df_SBA_All.append(df_SBA_2017[['FEMA Disaster Number','Damaged Property State Code','Verified Loss Real Estate','Approved Amount Real Estate']])
df_SBA_All = df_SBA_All.append(df_SBA_2018[['FEMA Disaster Number','Damaged Property State Code','Verified Loss Real Estate','Approved Amount Real Estate']])
df_SBA_All = df_SBA_All.append(df_SBA_2019[['FEMA Disaster Number','Damaged Property State Code','Verified Loss Real Estate','Approved Amount Real Estate']])
df_SBA_All = df_SBA_All.dropna().reset_index(drop=True)

df_SBA_All = df_SBA_All.rename(columns={"Damaged Property State Code": "State"})


# Get only the entries with an associated valid FEMA declaration number
cols = ['FEMA Disaster Number']
mask = df_SBA_All[cols].applymap(lambda x: isinstance(x, (int, float)))
df_SBA_All[cols] = df_SBA_All[cols].where(mask)
df_SBA_All['FEMA Disaster Number'] = df_SBA_All['FEMA Disaster Number'].astype(int).reset_index(drop=True)

# Get the ratio of loan-to-loss
df_SBA_All['Ratio'] = df_SBA_All['Approved Amount Real Estate']/df_SBA_All['Verified Loss Real Estate']

# If the loan approved is higher than the verified loss there is something wrong with this entry so I am removing it
# Using 1.05 to allow small adjustment to the loan amount 
df_SBA_All = df_SBA_All[df_SBA_All['Ratio'] <= 1.05].reset_index(drop=True)
print('There are',len(df_SBA_All.index),'entries in total.')

df_FEMA_Declarations = pd.read_csv(data_dir + "DisasterDeclarationsSummaries.csv")
df_FEMA_Declarations = df_FEMA_Declarations[df_FEMA_Declarations['fyDeclared']>2000]
df_FEMA_Declarations = df_FEMA_Declarations[df_FEMA_Declarations['declarationType'] == 'DR']

codes = df_FEMA_Declarations['disasterNumber']
years = df_FEMA_Declarations['fyDeclared']
FEMADeclaration_dict = dict(zip(codes, years))

hazard = df_FEMA_Declarations['incidentType']
FEMADeclarationHazard_dict = dict(zip(codes, hazard))

# Disaster year
df_SBA_All['Year'] = df_SBA_All['FEMA Disaster Number'].map(FEMADeclaration_dict)
df_SBA_All = df_SBA_All[df_SBA_All['Year'].isna() == False].reset_index(drop=True)

# Disaster hazard
df_SBA_All['Hazard'] = df_SBA_All['FEMA Disaster Number'].map(FEMADeclarationHazard_dict)
df_SBA_All['Hazard'] = df_SBA_All['Hazard'].replace(np.nan,'Unknown')

# Inflation rate since year
df_SBA_All['InflationRate'] = df_SBA_All['Year'].astype(int).apply(lambda x: cpi.inflate(1,x,to=2021))

There are 21759 entries in total.


## Get approval rate for selected state

In [45]:
def getApprovalRateForSelectedState(state):
    df_Approvals_state = df_SBA_All[df_SBA_All['Verified Loss Real Estate'] < 210000].copy()
     
    if state != 'All':  
        df_Approvals_state = df_Approvals_state[df_Approvals_state['State'] == state].copy()

    return len(df_Approvals_state[df_Approvals_state['Approved Amount Real Estate'] > 0].index) / len(df_Approvals_state.index)

## <font color='blue'> Use the cell below to calculate the approval rate per state</font>

In [46]:
# State of interest
theState = 'CA'
print('The approval rate for SBA loans in',theState,'is',getApprovalRateForSelectedState('CA'))

The approval rate for SBA loans in CA is 0.5408163265306123


# Approved amount

#### These cells manipulate the data, split aggregate data, and create a table of approval rates

In [47]:
# Get only approved loans because we are interested in those now
df_SBA_All = df_SBA_All[df_SBA_All['Approved Amount Real Estate'] > 0].reset_index(drop=True)


def getSplitLosses(theState,theHazard,df_AllLoans,df_FOIAApprovals):

    # Split loans between individual and aggregated
    if theState == 'All':
      df_ApprovedLoans = df_FOIAApprovals.copy()
      df_Ind = df_AllLoans[(df_AllLoans['Approved Amount Real Estate'] <= 210000) &\
                          (df_AllLoans['Hazard']==theHazard)].copy().reset_index(drop=True)

      df_Agg = df_AllLoans[(df_AllLoans['Approved Amount Real Estate'] > 210000) &\
                          (df_AllLoans['Hazard']==theHazard)].copy().reset_index(drop=True)

    elif theHazard == 'All':
      df_ApprovedLoans = df_FOIAApprovals[df_FOIAApprovals['STATE'] == theState].copy()
      df_Ind = df_AllLoans[(df_AllLoans['Approved Amount Real Estate'] <= 210000) &\
                          (df_AllLoans['State']==theState)].copy().reset_index(drop=True)

      df_Agg = df_AllLoans[(df_AllLoans['Approved Amount Real Estate'] > 210000) &\
                          (df_AllLoans['State']==theState)].copy().reset_index(drop=True)
    
    else: 
      df_ApprovedLoans = df_FOIAApprovals[df_FOIAApprovals['STATE'] == theState].copy()
      df_Ind = df_AllLoans[(df_AllLoans['Approved Amount Real Estate'] <= 210000) &\
                          (df_AllLoans['State']==theState) &\
                          (df_AllLoans['Hazard']==theHazard)].copy().reset_index(drop=True)

      df_Agg = df_AllLoans[(df_AllLoans['Approved Amount Real Estate'] > 210000) &\
                          (df_AllLoans['State']==theState) &\
                          (df_AllLoans['Hazard']==theHazard)].copy().reset_index(drop=True)


    # Fit an exponential RV to the approved loans in the FOIA data
    
    X = np.array(df_ApprovedLoans['ORIGINAL_APPROVAL_AMOUNT'], dtype=float)#df_ApprovedLoans['ORIGINAL_APPROVAL_AMOUNT']
    cut,mu = sts.expon.fit(X)


    # Return df
    df_return = pd.DataFrame()
    v_ratio = list(df_Ind['Ratio'])
    v_amt = list(df_Ind['Approved Amount Real Estate'])
    v_loss = list(df_Ind['Verified Loss Real Estate'])
    v_ddnumber = list(df_Ind['FEMA Disaster Number'])
    v_state = list(df_Ind['State'])
    v_year = list(df_Ind['Year'])
    v_hazard = list(df_Ind['Hazard'])


    # Loop over each aggregated entry
    for i in range(len(df_Agg.index)):

        # Get the value in this aggregated group
        totalLoan = df_Agg.loc[i,'Approved Amount Real Estate']

        while totalLoan > 0:

            # Estimate approved amount - note, the SBA cap is 200,000
            myApprovedAmount = min(np.random.exponential(mu,1)[0],200000)

            # Fit a multinomial RV to the Loan-to-Loss ratio for the individual loan data
            theBins = list(range(0,11,1))
            theBins = [x / 10 for x in theBins]

            # Get hold of the approved amount near the randomly assigned to this building
            df_Near = df_Ind[(df_Ind['Approved Amount Real Estate'] > myApprovedAmount - 50000)\
                                                    & (df_Ind['Approved Amount Real Estate'] < myApprovedAmount + 50000)].copy()

            count, division = np.histogram(df_Near[df_Near['Ratio']>0]['Ratio'],bins=theBins)
            p = count/np.sum(count)
            s = list(range(0,10,1))
            s = [x / 10 for x in s]

            # Estimate ratio within the 0.1-brackets
            myRatio = rng.choice(s,p=p,size=1)[0] + rng.random(1)[0]/10
            
            # Main calculations
            v_ratio.append(myRatio)
            v_amt.append(myApprovedAmount)
            v_loss.append(myApprovedAmount/myRatio)

            # Others
            v_ddnumber.append(df_Agg.loc[i,'FEMA Disaster Number'])
            v_state.append(df_Agg.loc[i,'State'])
            v_year.append(df_Agg.loc[i,'Year'])
            v_hazard.append(df_Agg.loc[i,'Hazard'])

            totalLoan -= myApprovedAmount

    df_return['FEMA Disaster Number'] = v_ddnumber
    df_return['State'] = v_state
    df_return['Verified Loss Real Estate'] = v_loss
    df_return['Approved Amount Real Estate'] = v_amt
    df_return['Ratio'] = v_ratio
    df_return['Year'] = v_year
    df_return['Hazard'] = v_hazard

    return df_return


def getApprovedAmountMatrixForSelectedState(state,hazard):

    df_sba = getSplitLosses(state,hazard,df_SBA_All,df_approvals)

    # Check in which bracket the losses for each applicant are
    out = pd.DataFrame()

    # Gather data for the selected state
    lossbins = list(range(0,500001,50000))
    lossbins.append(1000000)

    #print('The results are based on',len(df_State.index),'data points.')

    for i in range(len(lossbins)-1):
        df = df_sba[df_sba['Verified Loss Real Estate'].apply(lambda x: True if lossbins[i] <= x < lossbins[i+1] else False)]
        theBins = list(range(0,11,1))
        theBins = [x / 10 for x in theBins]
        count, division = np.histogram(df['Ratio'],bins=theBins)
        
        out[str(lossbins[i+1])] = count/np.sum(count)

    out.to_csv(data_dir+'SBA/ApprovalAmountMatrix_' + str(state) + '_' + str(hazard) +'.txt', index=False)
    return out

## <font color='blue'> Use the cell below to calculate the approval amounts per state and hazard </font>

In [48]:
# Enter 'All' if you do not want to filter by state or hazard
#                                        state,hazard
getApprovedAmountMatrixForSelectedState('All','Flood')

,50000,100000,150000,200000,250000,300000,350000,400000,450000,500000,1000000
0,0.010993,0.021624,0.028807,0.057211,0.072013,0.107843,0.113821,0.235294,0.223140,0.276596,0.455696
1,0.032978,0.055654,0.096022,0.117998,0.153846,0.166667,0.227642,0.209150,0.214876,0.234043,0.243671
2,0.065299,0.108118,0.137174,0.202622,0.180033,0.193627,0.223577,0.176471,0.214876,0.202128,0.231013
3,0.070878,0.102091,0.140604,0.157330,0.180033,0.176471,0.154472,0.150327,0.140496,0.170213,0.069620
4,0.073667,0.096420,0.126886,0.141836,0.176759,0.154412,0.130081,0.130719,0.206612,0.117021,0.000000
5,0.085316,0.101382,0.125514,0.114422,0.094926,0.100490,0.101626,0.098039,0.000000,0.000000,0.000000
6,0.112059,0.137185,0.129630,0.085816,0.055646,0.078431,0.048780,0.000000,0.000000,0.000000,0.000000
7,0.150943,0.137894,0.086420,0.047676,0.036007,0.022059,0.000000,0.000000,0.000000,0.000000,0.000000
8,0.155865,0.108827,0.056927,0.046484,0.044190,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
9,0.242002,0.130805,0.072016,0.028605,0.006547,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
